In [5]:
from upload_scripts import multipart_upload

In [ ]:
file = './data/utd19_u.csv'
bucket = 'utd19-u-raw'
key = 'measurements.csv'
chunk_size = 75

In [ ]:
multipart_upload(file=file, bucket=bucket, key=key, chunk_size=chunk_size)

Starting upload: C7qd5LstO21cfxfVZvaLWOMgL5GZwU47.7kPwvWvLYbyrYBKgPA51OUYMlA4t2z6Guse5mcKdNZ1f8cLUPinrCMvmxMemvKs42bKYTB_S4g0OPE_cxlWTlaDnm6X9Ey3
Finished Part: 1, ETag: "e50012f9b938928d599a739afa8f6fad"
Finished Part: 2, ETag: "421ec538dfc205608ca011e1bd8f30fd"
Finished Part: 4, ETag: "aa80a48284bc681410f97413ef2aadda"
Finished Part: 3, ETag: "68ae3c3e58a6f7a9a1154519eb8cef9b"
Finished Part: 8, ETag: "c7a9ae8619ecdecf1012c6bdade04b97"
Finished Part: 7, ETag: "85eb5bda337329a53ba678bac148e355"
Finished Part: 9, ETag: "6b7bbd19c426b2824ec753d2723f62a7"
Finished Part: 6, ETag: "212baff7d4d86fcc0d561c01d7f0d0b1"
Finished Part: 5, ETag: "6041f3d66866deae8cfd08194838d701"
Finished Part: 10, ETag: "081dcde82f6eb06ecd28f4250bee829b"
Finished Part: 11, ETag: "2409bcfa16033b9c24048fc3cb48fd09"
Finished Part: 12, ETag: "5126f1039ac8afbef6be2d8ed318a1e1"
Finished Part: 13, ETag: "c816236e5fffad732f7bd185b6cf1431"
Finished Part: 14, ETag: "790aff7416b86becda330192414276a3"
Finished Part: 16, ETag